In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('dupa').getOrCreate()

In [3]:
data = spark.read.csv(r"C:\Users\Asus\Desktop\studia big data\Python-and-Spark-for-Big-Data-master\Spark_for_Machine_Learning\Clustering\hack_data.csv", inferSchema=True, header=True)

In [14]:
data.columns

['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'Location',
 'WPM_Typing_Speed']

In [8]:
from pyspark.ml.feature import VectorAssembler 

In [15]:
assembler = VectorAssembler(inputCols=['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'WPM_Typing_Speed'], outputCol='features')

In [16]:
final_data = assembler.transform(data)

In [19]:
from pyspark.ml.feature import StandardScaler 

In [20]:
scaler = StandardScaler(inputCol='features', outputCol='stand_features')

In [21]:
scalerModel = scaler.fit(final_data)

In [22]:
final_data = scalerModel.transform(final_data)

In [23]:
from pyspark.ml.clustering import KMeans

In [29]:
kmean = KMeans(featuresCol='stand_features', k=3)

In [30]:
model = kmean.fit(final_data)

In [31]:
model.clusterCenters()

[array([2.99991988, 2.92319035, 1.05261534, 3.20390443, 4.51321315,
        3.28474   ]),
 array([1.30217042, 1.25830099, 0.        , 1.35793211, 2.57251009,
        5.24230473]),
 array([1.21780112, 1.37901802, 1.99757683, 1.37198977, 2.55237797,
        5.29152222])]

In [53]:
dupa = model.transform(final_data)

In [37]:
from pyspark.sql.functions import countDistinct

In [63]:
dupa.head(10)

[Row(Session_Connection_Time=8.0, Bytes Transferred=391.09, Kali_Trace_Used=1, Servers_Corrupted=2.96, Pages_Corrupted=7.0, Location='Slovenia', WPM_Typing_Speed=72.37, features=DenseVector([8.0, 391.09, 1.0, 2.96, 7.0, 72.37]), stand_features=DenseVector([0.5679, 1.3658, 1.9976, 1.2859, 2.2849, 5.3963]), prediction=2),
 Row(Session_Connection_Time=20.0, Bytes Transferred=720.99, Kali_Trace_Used=0, Servers_Corrupted=3.04, Pages_Corrupted=9.0, Location='British Virgin Islands', WPM_Typing_Speed=69.08, features=DenseVector([20.0, 720.99, 0.0, 3.04, 9.0, 69.08]), stand_features=DenseVector([1.4196, 2.518, 0.0, 1.3206, 2.9378, 5.151]), prediction=1),
 Row(Session_Connection_Time=31.0, Bytes Transferred=356.32, Kali_Trace_Used=1, Servers_Corrupted=3.71, Pages_Corrupted=8.0, Location='Tokelau', WPM_Typing_Speed=70.58, features=DenseVector([31.0, 356.32, 1.0, 3.71, 8.0, 70.58]), stand_features=DenseVector([2.2004, 1.2444, 1.9976, 1.6117, 2.6114, 5.2628]), prediction=2),
 Row(Session_Connectio

In [62]:
dupa.columns

['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'Location',
 'WPM_Typing_Speed',
 'features',
 'stand_features',
 'prediction']

In [64]:
schema = StructType([
    StructField("Session_Connection_Time", DoubleType(), True),
    StructField("Bytes Transferred", DoubleType(), True),
    StructField("Kali_Trace_Used", IntegerType(), True),
    StructField("Servers_Corrupted", DoubleType(), True),
    StructField("Pages_Corrupted", DoubleType(), True),
    StructField("Location", StringType(), True),
    StructField("WPM_Typing_Speed", DoubleType(), True),
    StructField("features", StringType(), True),
    StructField("stand_features", StringType(), True),
    StructField("prediction", IntegerType(), True)
])

NameError: name 'StructType' is not defined

In [65]:
from pyspark.sql.types import StructType, StructField, DoubleType, IntegerType, StringType

In [66]:
schema = StructType([
    StructField("Session_Connection_Time", DoubleType(), True),
    StructField("Bytes Transferred", DoubleType(), True),
    StructField("Kali_Trace_Used", IntegerType(), True),
    StructField("Servers_Corrupted", DoubleType(), True),
    StructField("Pages_Corrupted", DoubleType(), True),
    StructField("Location", StringType(), True),
    StructField("WPM_Typing_Speed", DoubleType(), True),
    StructField("features", StringType(), True),
    StructField("stand_features", StringType(), True),
    StructField("prediction", IntegerType(), True)
])

In [67]:
df = spark.createDataFrame(dupa, schema)

PySparkTypeError: [SHOULD_NOT_DATAFRAME] Argument `data` should not be a DataFrame.

In [70]:
dupa.groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|   84|
|         2|   83|
|         0|  167|
+----------+-----+

